In [1]:
import numpy as np
import json
import keras
import keras.preprocessing.text as kpt
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

from keras.initializers import glorot_normal
from keras import optimizers

/Users/martinfigueroapadilla/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
training = np.genfromtxt('data/tweets.csv', delimiter=',', skip_header=1, usecols=(1, 3), dtype=None)

train_x = [x[1] for x in training]
train_y = np.asarray([x[0] for x in training])
print(train_y)

[0 0 0 ... 2 2 2]


/Users/martinfigueroapadilla/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  """Entry point for launching an IPython kernel.


In [3]:
max_words = 10000

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(train_x)

dictionary = tokenizer.word_index
with open('dictionary.json', 'w') as dictionary_file:
    json.dump(dictionary, dictionary_file)

def convert_text_to_index_array(text):
    return [dictionary[word] for word in kpt.text_to_word_sequence(text)]

allWordIndices = []
for text in train_x:
    wordIndices = convert_text_to_index_array(text)
    allWordIndices.append(wordIndices)

allWordIndices = np.asarray(allWordIndices)

train_x = tokenizer.sequences_to_matrix(allWordIndices, mode='binary')
train_y = keras.utils.to_categorical(train_y, max(train_y) + 1)

In [4]:
out_dimension = 3
nHidden = 17
nInputDimensions = train_x.shape[1]

model = Sequential()
model.add(Dense(nHidden, input_dim=nInputDimensions, activation='elu', kernel_initializer=glorot_normal(seed=0)))

for i in range(0, 7):
    model.add(Dense(nHidden, activation='relu', kernel_initializer=glorot_normal(seed=0))) # agregar capa

model.add(Dense(out_dimension, activation='sigmoid', kernel_initializer=glorot_normal(seed=0)))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [6]:
model.fit(train_x, train_y, batch_size=5, epochs=100, verbose=1, validation_split=0.1, shuffle=True)

Train on 8373 samples, validate on 931 samples
Epoch 1/100
8373/8373 [==============================] - 14s 2ms/step - loss: 0.4418 - acc: 0.7857 - val_loss: 0.7480 - val_acc: 0.6223
Epoch 2/100
8373/8373 [==============================] - 13s 1ms/step - loss: 0.2319 - acc: 0.9026 - val_loss: 0.9614 - val_acc: 0.6076
Epoch 3/100
8373/8373 [==============================] - 14s 2ms/step - loss: 0.1523 - acc: 0.9363 - val_loss: 1.0435 - val_acc: 0.6627
Epoch 4/100
8373/8373 [==============================] - 14s 2ms/step - loss: 0.1120 - acc: 0.9524 - val_loss: 1.5274 - val_acc: 0.5961
Epoch 5/100
8373/8373 [==============================] - 13s 2ms/step - loss: 0.0932 - acc: 0.9576 - val_loss: 1.6363 - val_acc: 0.6491
Epoch 6/100
8373/8373 [==============================] - 14s 2ms/step - loss: 0.0779 - acc: 0.9639 - val_loss: 1.8009 - val_acc: 0.6728
Epoch 7/100
8373/8373 [==============================] - 14s 2ms/step - loss: 0.0721 - acc: 0.9664 - val_loss: 1.9964 - val_acc: 0.6527
E

In [7]:
model_json = model.to_json()
with open('model.h5', 'w') as json_file:
    json_file.write(model_json)

model.save_weights('weights.h5')